In [1]:
import yfinance as yf

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
import math

In [5]:
#import numpy as geek  

In [6]:
import matplotlib.pyplot as plt

In [7]:
import pandas_datareader as web

In [8]:
import statsmodels.api as sm

In [9]:
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date

In [10]:
pd.set_option('mode.use_inf_as_na', True)

In [11]:
stock = 'fb'

short_moving_average_span = 20
long_moving_average_span = 50
cutoff=0.50
invest = 100
years_of_data_to_process = 25
period = 15

In [12]:
currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
#print (date)
year = date.strftime("%Y")
start_year =  int(year) - years_of_data_to_process
start = datetime.datetime(start_year, 1, 1)

In [ ]:
data =  yf.download(stock, start=start)

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\yfinance\multi.py", line 167, in _download_one_threaded
    data = _download_one(ticker, start, end, auto_adjust, back_adjust,
  File "C:\ProgramData\Anaconda3\lib\site-packages\yfinance\multi.py", line 179, in _download_one
    return Ticker(ticker).history(period=period, interval=interval,
  File "C:\ProgramData\Anaconda3\lib\site-packages\yfinance\base.py", line 157, in history
    data = data.json()
  File "C:\ProgramData\Anaconda3\lib\site-packages\requests\models.py", line 898, in json
    return complexjson.loads(self.t

In [ ]:
#data = web.DataReader(stock, data_source='yahoo', start = start, end = date)

In [ ]:
df = data["Close"].pct_change() * 100

In [ ]:
df = df.rename("Today_Change_%")
df = df.reset_index()

In [ ]:
df1 = pd.merge(data,df, on="Date")
#df1

In [ ]:
#df1.dropna(inplace= True)
df1.insert(7,'Volume_Lag', None)
df1.Volume = df1.Volume.pct_change() * 100
#df1.Volume = df1.Volume *  geek.sign(df1.Volume.values)
df1['Volume_Lag'] = df1.Volume.shift(1).values


In [ ]:
df1["Trend"] =  (df1["Close"] - df1["Low"])/ ((df1["High"] - df1["Low"]))

In [ ]:
df1["Trend_Lag"] = df1["Trend"].shift(1)

In [ ]:
df1['Close-Open'] =  df1['Close'] - df1['Open']
df1['Close-Open_Lag'] = df1['Close-Open'].shift(1)

In [ ]:
df1['High-Low'] =  df1['High'] - df1['Low']
df1['High-Low_Lag'] = df1['High-Low'].shift(1)

In [ ]:
df1 = sm.add_constant(df1)
df1["Up_Down"] = [1 if (i > 0) else 0 for i in df1["Today_Change_%"]]

In [ ]:
df1.dropna(inplace= True)

In [ ]:
df1['Short_MV_Avg_Span'] = df1['Close'].ewm(span=short_moving_average_span, adjust=False).mean()
df1.dropna(inplace= True)
#df1

In [ ]:
df1['Long_MV_Avg_Span'] = df1['Close'].ewm(span=long_moving_average_span, adjust=False).mean()
df1.dropna(inplace= True)
#df1

In [ ]:
df1['Short_MV_Avg_Span-Long_MV_Avg_Span'] = df1.Short_MV_Avg_Span - df1.Long_MV_Avg_Span
df1['Short_MV_Avg_Span-Long_MV_Avg_Span_Lag'] = df1['Short_MV_Avg_Span-Long_MV_Avg_Span'].shift(1)
#df1

In [ ]:
df1.dropna(inplace= True)
df1.tail(10)

In [ ]:
#X = df1[['const','Trend_Lag','Short_MV_Avg_Span-Long_MV_Avg_Span_Lag','Close-Open_Lag','High-Low_Lag','Volume_Lag']]

In [ ]:
#df1['Close'][-150:].plot(figsize=(16,6))
#df1['Short_MV_Avg_Span'][-150:].plot(figsize=(16,6))
#df1['Long_MV_Avg_Span'][-150:].plot(figsize=(16,6))
#plt.xlim([len(df1)-100, len(df1)])
#plt.xlabel('Index', fontsize=18)
#plt.ylabel('Close Price USD ($)', fontsize =18)
#plt.legend(['Close', str(short_moving_average_span) + ' Days Moving Average', str(long_moving_average_span) + ' days Moving Average'], loc = 'lower left')
#plt.title("Moving Average for " + stock.upper(), fontsize = 16)
#plt.show
#plt.close

In [ ]:
fig, ax1 = plt.subplots()
ax2 =  ax1.twinx()
df1['Close'][-150:].plot(x = 'Index', color='tab:blue', figsize=(16,6), label = 'Close Price USA ($)', fontsize = 12, ax = ax1)
df1['Short_MV_Avg_Span'][-150:].plot(x = 'Index', color='tab:red', figsize=(16,6), label = str(short_moving_average_span) + ' Days Moving Average', fontsize = 12, ax = ax1)
df1['Long_MV_Avg_Span'][-150:].plot(x = 'Index', color='tab:green', figsize=(16,6), label = str(long_moving_average_span) + ' days Moving Average', fontsize = 12, ax = ax1)
df1['Short_MV_Avg_Span-Long_MV_Avg_Span_Lag'][-150:].plot(x = 'Index', color='tab:brown', figsize=(16,6), label = 'Signal_line', fontsize = 12, ax = ax2)
#df1['RSI'][-170:].plot(x = 'Index', color='tab:red', figsize=(16,6), label = 'Relative Strength Index', fontsize = 12,ax = ax1)
#df1['Close'][-150:].plot(x = 'Index',color = 'tab:blue', figsize=(16,6),  label = 'Close Price',fontsize = 12,ax = ax2)
#df1['Close'][-100:].plot(figsize=(16,6))
plt.xlim([len(df1)-120, len(df1)])
ax1.set_ylabel('Close Price USD ($)', fontsize = 18)
ax2.set_ylabel('Signal_line', fontsize = 18)
#df1.xlabel('Index', fontsize=18)
#df1.ylabel('Relative Strength Index', fontsize =18)
ax2.grid(color='tab:brown')
ax1.set_title("Moving Average & Signal Line for " + stock.upper(), fontsize = 16)
ax1.legend(loc=2, fontsize = 16)
ax2.legend(loc=4,fontsize = 16)
plt.show
plt.close

In [ ]:
df1['Signal_Line'] = df1['Short_MV_Avg_Span-Long_MV_Avg_Span_Lag'].ewm(span = period, adjust=False ).mean()
#column_list = list(df1)
#print (column_list)
#df1=df1[['const', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Volume_Lag', 'Today_Change_%', 'Trend', 'Trend_Lag', 'Close-Open', 'Close-Open_Lag', 'High-Low', 'High-Low_Lag', 'Up_Down', 'Short_MV_Avg_Span', 'Long_MV_Avg_Span', 'Short_MV_Avg_Span-Long_MV_Avg_Span', 'Short_MV_Avg_Span-Long_MV_Avg_Span_Lag', 'Prediction_Caculated', 'Prediction_indicator', 'share', 'money', 'Up', 'Down', 'Signal_Line']]
df1['Signal_Line_Lag'] =  df1['Signal_Line'].shift(1)
df1.dropna(inplace= True)
#df1.info()

In [ ]:
X = df1[['const','Trend_Lag','Signal_Line_Lag','Volume_Lag']]

In [ ]:
y = df1["Up_Down"].values

In [ ]:
model = sm.Logit(y,X)

In [ ]:
result =  model.fit()

In [ ]:
result.summary()

In [ ]:
prediction = result.predict(X)

In [ ]:
df1['Prediction_Caculated'] = pd.array(prediction)
df1['Prediction_indicator'] = pd.array([1 if i > cutoff else 0 for i in prediction])
#print (df1.iloc[-1,18],df1.iloc[-1,19],df1.iloc[-1,20])

In [ ]:
#print(df1.loc[[i for i in range(len(df1)-20, len(df1)+2)],['Trend_Lag', 'Short_MV_Avg_Span-Long_MV_Avg_Span_Lag', 'Up_Down', 'Prediction_indicator']])

In [ ]:
y = df1["Up_Down"].values

In [ ]:
def confusion_matrix(act,pred):
    predtrans = ['Up' if i > cutoff else 'Down' for i in pred]
    actuals = ['Up' if i > 0 else 'Down' for i in act]
    confusion_matrix = pd.crosstab(pd.Series(actuals),
                                   pd.Series(predtrans),
                                   rownames = ["Actual"],
                                   colnames = ["Predict"]
                                  )
    return confusion_matrix

In [ ]:
confusion_matrix(y,prediction)

In [ ]:
z = confusion_matrix(y,prediction)
try:
    print((z.loc['Down','Down'] + z.loc['Up','Up']) / len(df1))
except:
    pass

In [ ]:
try:
    print( (z.loc['Down', 'Down']+ z.loc['Up','Up']) / (z.loc['Down', 'Down']+ z.loc['Up','Up'] + z.loc['Down','Up']) )
except:
    pass

In [ ]:
df1 = df1.assign(share=np.nan,money=np.nan)
#df1

In [ ]:
diff_years = round((df1.iloc[-1,1] - df1.iloc[0,1])/np.timedelta64(1,'Y') + 0.5)   

In [ ]:
#df1.info()
#column_list = list(df1)
#print (column_list)
df1['Signal_Line_Lag'] = df1['Signal_Line'].shift(1)
df1=df1[['const', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Volume_Lag', 'Today_Change_%', 'Trend', 'Trend_Lag', 'Close-Open', 'Close-Open_Lag', 'High-Low', 'High-Low_Lag', 'Up_Down', 'Short_MV_Avg_Span', 'Long_MV_Avg_Span', 'Short_MV_Avg_Span-Long_MV_Avg_Span', 'Short_MV_Avg_Span-Long_MV_Avg_Span_Lag', 'Prediction_Caculated', 'Prediction_indicator', 'share', 'money', 'Signal_Line', 'Signal_Line_Lag']]
#df1.info()

In [ ]:
#Simulate Investment transaction buy on opening when predict UP and sell daily average when predict DOWN

def buy_sell(open_price, sell_price,prediction, money, share):
    if prediction == 1 and money != 0:
        share =  money / open_price
        money = 0
    elif prediction == 0 and share != 0:
        money = share * sell_price
        share = 0
    else: pass
    return [money, share]
money = invest
share = 0
for i in range(len(df1)):
    [money, share] = buy_sell(df1.iloc[i,2],(df1.iloc[i,3]+df1.iloc[i,4])/2,df1.iloc[i,22], money, share)
    df1.iloc[i,23] = share
    df1.iloc[i,24] = money

In [ ]:
#df1.info()

In [ ]:
df1 = df1.assign(Up=np.nan,Down=np.nan)
#df1.info()

In [ ]:
for i in range(len(df1)):
    if df1.iloc[i,9] <= 0:
        df1.iloc[i,27] = 0
        df1.iloc[i,28] = df1.iloc[i,9]
    else:
        df1.iloc[i,28] = 0
        df1.iloc[i,27] = df1.iloc[i,9]

AVG_Gain = df1.Up.ewm(span=period, adjust=False).mean()
AVG_Loss = df1.Down.ewm(span=period, adjust=False).mean().abs()

In [ ]:
RS = AVG_Gain /AVG_Loss
RSI = 100.0 - (100.0 / (1.0 + RS))
df1['RSI'] = RSI
df1['RSI_Lag'] = df1['RSI'].shift(1)
#df1.tail(10)

In [ ]:
fig, ax1 = plt.subplots()
ax2 =  ax1.twinx()
df1['RSI'][-170:].plot(x = 'Index', color='tab:red', figsize=(16,6), label = 'Relative Strength Index', fontsize = 12,ax = ax2)
df1['Close'][-150:].plot(x = 'Index',color = 'tab:blue', figsize=(16,6),  label = 'Close Price',fontsize = 12,ax = ax1)
#df1['Close'][-100:].plot(figsize=(16,6))
plt.xlim([len(df1)-100, len(df1)])
ax2.set_ylabel('Relative Strength Index', fontsize = 18)
ax1.set_ylabel('Close Price USD ($)', fontsize = 18)
#df1.xlabel('Index', fontsize=18)
#df1.ylabel('Relative Strength Index', fontsize =18)
ax2.grid(color='tab:red')
ax1.set_title("Relative Strength Index for " + stock.upper(), fontsize = 16)
ax2.legend(loc=3, fontsize = 16)
ax1.legend(loc=2,fontsize = 16)
plt.show
plt.close

In [ ]:
fig, ax1 = plt.subplots()
ax2 =  ax1.twinx()
df1['Trend_Lag'][-150:].ewm(span = period *2, adjust=False ).mean().plot(x = 'Index', color='tab:red', figsize=(16,6), label = 'Buying Trend', fontsize = 12,ax = ax2)
df1['Close'][-150:].plot(x = 'Index',color = 'tab:blue', figsize=(16,6),  label = 'Close Price',fontsize = 12,ax = ax1)
ax2.set_ylabel('Buying Trend', fontsize = 18)
ax1.set_ylabel('Close Price USD ($)', fontsize = 18)
plt.xlim([len(df1)-100, len(df1)])
ax2.grid(color='tab:red')
ax1.set_title("Buying Trend for " + stock.upper(), fontsize = 16)
ax2.legend(loc=3, fontsize = 16)
ax1.legend(loc=2,fontsize = 16)
plt.close

In [ ]:
df1.dropna(inplace= True)
X = df1[['const','Trend_Lag', 'RSI_Lag', 'Signal_Line_Lag','Volume_Lag']]
y = df1["Up_Down"].values
model = sm.Logit(y,X)
result =  model.fit()
result.summary()

In [ ]:
print("\nIf ${:,.0f} was invested in [ {} ], and Just Hold and Not Trade for {:2d} years, the ROI = ${:,.0f}".format( invest, stock.upper(), diff_years, invest/data.iloc[0, 0] * data.iloc[-1,0]))

In [ ]:
print ("\nIf ${:,.0f} was invested {:2d} years ago, buy and sell according this script\'s recommandation, the ROI = ${:,.0f}".format(invest, diff_years, (money + (share * df1.iloc[-1,5]))))

In [ ]:
df1_summary=df1[['Date', 'Up_Down','Prediction_indicator']].copy()
df1_summary['Stock Market Performance'] = df1_summary['Up_Down'].apply(lambda x: 'Up' if x > 0 else 'Down')
df1_summary['Scribe Predection'] = df1_summary['Prediction_indicator'].apply(lambda x: 'Up' if x > 0 else 'Down')
print (df1_summary[['Date','Stock Market Performance','Scribe Predection']].tail(15))

In [ ]:
#print(df1.loc[[i for i in range(len(df1)-20, len(df1)+2)],['Date','Close','Trend_Lag', 'Short_MV_Avg_Span-Long_MV_Avg_Span_Lag']])

In [ ]:
print ("\nToday [ %s ] actually went up," %stock.upper(), end = ' ') if (df1.iloc[-1,16] == 1) else print ("\nToday [ %s ] actually went down," %stock.upper(), end = " ")
print ("--- base on yesterday\'s data, ", end = '')
print ("We Predication [ %s ] should be going up." %stock.upper()) if (df1.iloc[-1,22] == 1) else print ("We Predicae [ %s ] should be going down." %stock.upper())
print ("\n=========> Actual and Predication MATCH <=========") if (df1.iloc[-1,16] == df1.iloc[-1,22]) else print("\n=========> Actual and Predication DO NOT match <=========")

In [ ]:
x_tran= df1[df1.Date.dt.year < 2021][['const','Trend_Lag', 'RSI_Lag','Signal_Line_Lag','Volume_Lag']]
y_train=df1[df1.Date.dt.year < 2021]["Up_Down"]
x_test= df1[df1.Date.dt.year >= 2021][['const','Trend_Lag', 'RSI_Lag','Signal_Line_Lag','Volume_Lag']]
y_test= df1[df1.Date.dt.year >= 2021]["Up_Down"]

In [ ]:
model = sm.Logit(y_train,x_tran)
result=model.fit()

In [ ]:
result.summary()

In [ ]:
#summary_list=result.summary().as_csv().split(",")

In [ ]:
#print(summary_list)

In [ ]:
#print(summary_list[37:39])

In [ ]:
prediction = result.predict(x_test)
confusion_matrix(y_test, prediction)

In [ ]:
z = confusion_matrix(y_test,prediction)

In [ ]:
try:
    print ("\n=========> Prediction Accuracy Rate: %.4f <=========\n"  %((z.loc['Down','Down'] + z.loc['Up','Up']) / len(x_test)))
except:
    print ("\n=========> Predication effectiveness is not avairable <=========\n" )

In [ ]:
#df1.info()

In [ ]:
prediction = result.predict(x_test)
now_up_down  = result.predict([1.0, df1.iloc[-1, 10], df1.iloc[-1, 28], df1.iloc[-1, 21], df1.iloc[-1, 7]])
print ("\n=========> Current trend = %.4f,  " %now_up_down, end=' ')
print ("[ %s ] will go up! <=========" %stock.upper()) if now_up_down > cutoff else print ("[ %s ] will go down! <=========" %stock.upper()) 

In [ ]:
#print ("\n=========> Current trend = %.4f,  " %now_up_down, end=' ')
print ("\n ============> %s Days over %s Days Moving Average Indicator \n ============> %.4f....%.4f....%.4f....%.4f....%.4f....<=============" %(short_moving_average_span, long_moving_average_span, df1.iloc[-5,19], df1.iloc[-4,19], df1.iloc[-3,19], df1.iloc[-2,19],df1.iloc[-1,19]))
if df1.iloc[-1,19] * df1.iloc[-2,19] < 0:
    print ("\n ============> Warning, It Is the Time to Sell [ %s ] <=========" %stock.upper()) if df1.iloc[-1,19] < 0 else print ("\n ============> It Is the Time to Buy [ %s ] ! <=========" %stock.upper())
else:
    print ("\n ============> No Trading Waring at this time! <=============")

In [ ]:
#df1.to_csv('fb.csv', index = False)